In [5]:
# !pip install -U voyageai pandas
!pip install clarifai-grpc

In [6]:
import os
from typing import Dict, List, Optional
import glob
import json
from tqdm import tqdm
import time
import dotenv
import traceback
import base64
from clarifai_grpc.channel.clarifai_channel import ClarifaiChannel
from clarifai_grpc.grpc.api import service_pb2_grpc, service_pb2, resources_pb2
from clarifai_grpc.grpc.api.status import status_code_pb2

def classify_apparel_images(
    directories: List[str],
    max_images_per_dir: Dict[str, int],
    pat: str,
    user_id: str = "clarifai",
    app_id: str = "main",
    model_id: str = "apparel-classification-v2",
    model_version_id: str = "651c5412d53c408fa3b4fe3dcc060be7",
    max_concepts: int = 5,  # New parameter to specify maximum number of concepts to return
    output_file: Optional[str] = "classification_results.json",
) -> Dict:
    """
    Classify apparel images in specified directories using Clarifai API.

    Args:
        directories: List of directory names containing apparel images
        max_images_per_dir: Dictionary mapping directory names to maximum number of images to process
        pat: Clarifai Personal Access Token
        user_id: Clarifai user ID
        app_id: Clarifai app ID
        model_id: Clarifai model ID
        model_version_id: Optional model version ID (defaults to latest if None)
        max_concepts: Maximum number of concepts to return per image
        output_file: Optional file path to save results (None to skip saving)

    Returns:
        Dictionary containing classification results for each image
    """
    print(f"Using model: user_id={user_id}, app_id={app_id}, model_id={model_id}, version={model_version_id}")
    
    # Set up the gRPC client
    channel = ClarifaiChannel.get_grpc_channel()
    stub = service_pb2_grpc.V2Stub(channel)
    metadata = (('authorization', 'Key ' + pat),)
    user_data_object = resources_pb2.UserAppIDSet(user_id=user_id, app_id=app_id)

    # Dictionary to store results
    results = {}

    # Process each directory
    for directory in directories:
        print(f"Processing {directory} directory...")

        # Get image paths in the directory
        image_pattern = os.path.join(directory, "*.jpg")
        image_paths = glob.glob(image_pattern)

        # Determine how many images to process
        max_images = max_images_per_dir.get(directory, 10)  # Default to 10 if not specified
        image_paths = image_paths[:max_images]

        # Process each image
        for image_path in tqdm(image_paths):
            try:
                # Read image file as bytes
                with open(image_path, "rb") as f:
                    image_bytes = f.read()
                
                # Create the request with file bytes instead of URL
                post_model_outputs_response = stub.PostModelOutputs(
                    service_pb2.PostModelOutputsRequest(
                        user_app_id=user_data_object,
                        model_id=model_id,
                        version_id=model_version_id,
                        inputs=[
                            resources_pb2.Input(
                                data=resources_pb2.Data(
                                    image=resources_pb2.Image(
                                        base64=image_bytes
                                    )
                                )
                            )
                        ]
                    ),
                    metadata=metadata
                )

                # Check for errors in the API response
                if post_model_outputs_response.status.code != status_code_pb2.SUCCESS:
                    error_msg = f"API Error: {post_model_outputs_response.status.description}"
                    print(error_msg)
                    results[image_path] = f"Error: {error_msg}"
                    continue

                # Get the output from the response
                output = post_model_outputs_response.outputs[0]
                
                # Extract top concepts
                if len(output.data.concepts) > 0:
                    # Sort concepts by value (confidence score) in descending order
                    concepts = sorted(output.data.concepts, key=lambda c: c.value, reverse=True)
                    
                    # Get up to max_concepts or all available if fewer
                    top_concepts = concepts[:max_concepts]
                    
                    # Format the classifications
                    classifications = [f"{c.name} ({c.value:.2f})" for c in top_concepts]
                    
                    # Join with semicolons for better readability
                    classification_str = "; ".join(classifications)
                else:
                    classification_str = "No concepts found"

                # Store result
                results[image_path] = classification_str
                print(f"Success: {image_path} - {classification_str}")

            except Exception as e:
                # Get detailed error information
                error_details = traceback.format_exc()
                print(f"Error processing {image_path}:")
                print(error_details)
                results[image_path] = f"Error: {str(e)}"

            # Add a delay to avoid rate limiting
            time.sleep(2.0)

    # Save results to file if specified
    if output_file:
        with open(output_file, "w") as f:
            json.dump(results, f, indent=2)
        print(f"Results saved to {output_file}")

    return results

def process_classification_results(results_file: str) -> dict:
    """
    Process classification results to extract only the item names without confidence scores.
    
    Args:
        results_file: Path to the JSON file containing classification results
        
    Returns:
        Dictionary with image paths as keys and lists of item names as values
    """
    # Load the results from the JSON file
    with open(results_file, 'r') as f:
        results = json.load(f)
    
    # Dictionary to store processed results
    processed_results = {}
    
    # Process each image result
    for image_path, classifications in results.items():
        # Skip error entries
        if classifications.startswith("Error") or classifications == "No concepts found":
            processed_results[image_path] = []
            continue
        
        # Split classifications by semicolon
        items_with_scores = classifications.split('; ')
        
        # Extract just the item names (remove the confidence scores)
        items = []
        for item_with_score in items_with_scores:
            # Extract the item name (everything before the opening parenthesis)
            item_name = item_with_score.split(' (')[0]
            items.append(item_name)
        
        # Store in the processed results dictionary
        processed_results[image_path] = items
    
    # Save the processed results to a new JSON file
    output_file = 'processed_clarifai_classifications.json'
    with open(output_file, 'w') as f:
        json.dump(processed_results, f, indent=2)
    
    print(f"Processed results saved to {output_file}")
    
    return processed_results


def main():
    # Load environment variables from .env file
    dotenv.load_dotenv()

    # Get PAT from environment variable
    pat = os.environ.get("PAT")

    if not pat:
        print(
            "Error: PAT environment variable not found. Please set PAT in your .env file."
        )
        return None

    print(
        f"Using PAT: {pat[:5]}...{pat[-4:] if len(pat) > 8 else ''} (masked for security)"
    )

    directories = ["same_color"]

    # Process all images in the directory by setting a very high number
    max_images_per_dir = {
        "same_color": 1000  # This high number ensures we process all images
    }

    output_file = "clarifai_classification_results.json"
    
    # Classify images
    results = classify_apparel_images(
        directories=directories,
        max_images_per_dir=max_images_per_dir,
        pat=pat,
        max_concepts=5,  # Get up to 5 classifications per image
        output_file=output_file,
    )

    # Process the results to extract just the item names
    processed_results = process_classification_results(output_file)
    
    return processed_results


# Process existing results file without running classification again
# processed_results = process_classification_results("clarifai_classification_results.json")
# processed_results

results = main()



Using PAT: 54e90...d229 (masked for security)
Using model: user_id=clarifai, app_id=main, model_id=apparel-classification-v2, version=651c5412d53c408fa3b4fe3dcc060be7
Processing same_color directory...


  0%|          | 0/3 [00:00<?, ?it/s]

Success: same_color/yellow.jpg - coat (0.97); long-sleeve (0.94); 3/4 sleeve (0.67); chiffon (0.62); midi dress (0.54)


 33%|███▎      | 1/3 [00:07<00:14,  7.37s/it]

Success: same_color/green.jpg - coat (0.98); long-sleeve (0.85); 3/4 sleeve (0.76); midi dress (0.55); colorblock (0.50)


 67%|██████▋   | 2/3 [00:09<00:04,  4.55s/it]

Success: same_color/purple.jpg - coat (0.99); long-sleeve (0.92); v-neck (0.63); midi dress (0.62); 3/4 sleeve (0.54)


100%|██████████| 3/3 [00:12<00:00,  4.17s/it]

Results saved to clarifai_classification_results.json
Processed results saved to processed_clarifai_classifications.json


In [ ]:
import os
import json
import base64
from anthropic import Anthropic

def initialize_anthropic_client():
    api_key = os.environ.get("ANTHROPIC_API_KEY")
    if not api_key:
        print("Error: ANTHROPIC_API_KEY not found in environment variables.")
        return None
    return Anthropic(api_key=api_key)

def analyze_attribute_with_claude(client, image_path, attribute):
    """Analyze a single attribute of a garment using Claude."""
    try:
        with open(image_path, "rb") as image_file:
            encoded_string = base64.b64encode(image_file.read()).decode('utf-8')
    except FileNotFoundError:
        print(f"Error: Image file not found at path: {image_path}")
        return {"error": "Image file not found."}
    
    messages = [
        {
            "role": "user",
            "content": [
                {
                    "type": "image",
                    "source": {
                        "type": "base64",
                        "media_type": "image/jpeg",
                        "data": encoded_string
                    }
                },
                {
                    "type": "text",
                    "text": f"""For this garment, focus only on the '{attribute}' aspect. 
                    Return a JSON object with: color, material, occasion, style, season, unique_feature, era, 
                    casual_or_relaxed (boolean), visual_aesthetic, hardware."""
                }
            ]
        }
    ]
    
    try:
        response = client.messages.create(
            model="claude-3-7-sonnet-latest",
            max_tokens=800,
            system="You are a fashion analyst focused on specific garment attributes.",
            messages=messages
        )
        
        response_text = response.content[0].text
        json_start = response_text.find('{')
        json_end = response_text.rfind('}') + 1
        
        if json_start >= 0 and json_end > json_start:
            json_content = response_text[json_start:json_end]
            try:
                return json.loads(json_content)
            except json.JSONDecodeError:
                return {"error": "Failed to parse JSON response"}
        else:
            return {"error": "No JSON in response"}
            
    except Exception as e:
        return {"error": f"Error processing image: {e}"}

def describe_clothing_item_attributes(json_path, base_img_dir=""):
    client = initialize_anthropic_client()
    if not client:
        return {}
    
    try:
        with open(json_path, 'r') as file:
            processed_results = json.load(file)
    except (FileNotFoundError, json.JSONDecodeError) as e:
        print(f"Error loading JSON: {e}")
        return {}
    
    detailed_descriptions = {}
    
    for image_path, attributes in processed_results.items():
        full_image_path = os.path.join(base_img_dir, image_path) if base_img_dir else image_path
        
        # Create a list to store descriptions for each attribute
        attribute_descriptions = []
        
        for attribute in attributes:
            print(f"Processing {image_path} - {attribute}")
            
            # Get detailed analysis from Claude for this specific attribute
            attribute_details = analyze_attribute_with_claude(client, full_image_path, attribute)
            
            # Add the attribute name to the details
            attribute_details["attribute"] = attribute
            attribute_descriptions.append(attribute_details)
        
        # Store the list of attribute descriptions for this image
        detailed_descriptions[image_path] = attribute_descriptions
    
    return detailed_descriptions

def link_with_anthropic_descriptors():
    json_path = "processed_clarifai_classifications.json"
    base_img_dir = ""  # Set if your images are in a different directory
    
    results = describe_clothing_item_attributes(json_path, base_img_dir)
    
    with open("anthropic_clarifai_descriptions.json", "w") as f:
        json.dump(results, f, indent=2)
    
    print(f"Analysis complete. Results saved to anthropic_clarifai_descriptions.json")

anthro_result = link_with_anthropic_descriptors()
anthro_result

Processing same_color/yellow.jpg - coat
Processing same_color/yellow.jpg - long-sleeve
Processing same_color/yellow.jpg - 3/4 sleeve
Processing same_color/yellow.jpg - chiffon
Processing same_color/yellow.jpg - midi dress
Processing same_color/green.jpg - coat
Processing same_color/green.jpg - long-sleeve
Processing same_color/green.jpg - 3/4 sleeve
Processing same_color/green.jpg - midi dress
Processing same_color/green.jpg - colorblock
Processing same_color/purple.jpg - coat
Processing same_color/purple.jpg - long-sleeve
Processing same_color/purple.jpg - v-neck
Processing same_color/purple.jpg - midi dress
Processing same_color/purple.jpg - 3/4 sleeve
Analysis complete. Results saved to detailed_attribute_descriptions.json


In [ ]:
import voyageai
import pandas as pd
import os
import json
import time
from dotenv import load_dotenv

# Load environment variables (API key)
load_dotenv(override=True)
VOYAGE_API_KEY = os.getenv("VOYAGE_API_KEY")

# Initialize Voyage client with API key
vo = voyageai.Client(api_key=VOYAGE_API_KEY)

def load_attribute_descriptions(json_file_path):
    """Load the attribute descriptions from a JSON file."""
    with open(json_file_path, 'r') as f:
        return json.load(f)

def prepare_text_for_embedding(attribute_list):
    """Convert attribute dictionary list to a single text string for embedding."""
    text_parts = []
    
    for attr_dict in attribute_list:
        attribute = attr_dict.get('attribute', 'unknown')
        text_parts.append(f"Attribute: {attribute}")
        
        # Add all other fields
        for key, value in attr_dict.items():
            if key != 'attribute' and key != 'error':
                text_parts.append(f"{key}: {value}")
        
        text_parts.append("---")  # Separator between attributes
    
    return " ".join(text_parts)

def embed_fashion_descriptions(json_file_path, output_file="embeddings_same_color.csv"):
    """Generate embeddings for each fashion item's full attribute description."""
    # Load the attribute descriptions
    descriptions = load_attribute_descriptions(json_file_path)
    
    # Prepare DataFrame to store results
    results = []
    
    # Process each image path
    for image_path, attributes in descriptions.items():
        print(f"Processing: {image_path}")
        
        # Convert the list of attribute dictionaries to a single text
        full_description = prepare_text_for_embedding(attributes)
        
        try:
            # Generate embedding for the entire dictionary as one text
            embedding_result = vo.embed(
                texts=[full_description],
                model="voyage-3-large",
                input_type="document",
                output_dimension=256,
                output_dtype="float"
            )
            
            # Store the result
            results.append({
                'image_path': image_path,
                'embedding': embedding_result.embeddings[0],
                'raw_attributes': attributes  # Store original attributes for reference
            })
            
            print(f"Successfully embedded {image_path}")
            
        except Exception as e:
            print(f"Error embedding {image_path}: {e}")
        
        # Add a small delay to respect rate limits
        time.sleep(3)
    
    # Convert results to DataFrame
    df = pd.DataFrame(results)
    
    # Save to CSV (embeddings will be stored as strings)
    df_to_save = df.copy()
    df_to_save['embedding'] = df_to_save['embedding'].apply(lambda x: ','.join(map(str, x)))
    df_to_save['raw_attributes'] = df_to_save['raw_attributes'].apply(json.dumps)
    df_to_save.to_csv(output_file, index=False)
    
    print(f"Saved embeddings to {output_file}")
    return df

# Function to search similar items using the embeddings
def find_similar_fashion_items(df, query_image_path=None, query_text=None, top_n=5):
    """Find similar fashion items based on embeddings similarity."""
    if query_image_path is not None and query_image_path in df['image_path'].values:
        # Get the embedding for the query image
        query_embedding = df[df['image_path'] == query_image_path]['embedding'].iloc[0]
    elif query_text is not None:
        # Generate embedding for the query text
        try:
            query_result = vo.embed(
                texts=[query_text],
                model="voyage-3-large",
                input_type="document",
                output_dimension=256,
                output_dtype="float"
            )
            query_embedding = query_result.embeddings[0]
        except Exception as e:
            print(f"Error generating embedding for query text: {e}")
            return None
    else:
        print("Either query_image_path or query_text must be provided")
        return None
    
    # Calculate similarity scores
    df['similarity'] = df['embedding'].apply(
        lambda emb: sum(a*b for a, b in zip(emb, query_embedding)) / 
        (sum(a*a for a in emb)**0.5 * sum(b*b for b in query_embedding)**0.5)
    )
    
    # Sort by similarity and return top matches
    return df.sort_values('similarity', ascending=False).head(top_n)

# Example usage in a Jupyter notebook
# Run this in a cell

# Load and embed fashion descriptions
json_file_path = "anthropic_same_color_descriptions.json"
embeddings_same_color_df = embed_fashion_descriptions(json_file_path, output_file="embeddings_same_color.csv")
embeddings_same_color_df.head(2)


Processing: same_color/yellow.jpg
Successfully embedded same_color/yellow.jpg
Processing: same_color/green.jpg
Successfully embedded same_color/green.jpg
Processing: same_color/purple.jpg
Successfully embedded same_color/purple.jpg


In [ ]:
import phoenix as px
import pandas as pd
import numpy as np
import ast

# Load the fashion embeddings data
fashion_df = pd.read_csv('fashion_embeddings.csv')

# Convert embedding strings to numpy arrays
fashion_df['embedding'] = fashion_df['embedding'].apply(lambda x: np.array([float(val) for val in x.split(',')]))

# Convert raw_attributes from string to dictionaries
def parse_attributes(attr_str):
    try:
        # First, try parsing as a list of dictionaries
        attributes = ast.literal_eval(attr_str)
        # If it's a list, return the first item (or combine them if needed)
        if isinstance(attributes, list):
            return attributes[0]  # Alternatively, you could merge all dictionaries
        return attributes
    except:
        # If parsing fails, return an empty dictionary
        return {}

fashion_df['parsed_attributes'] = fashion_df['raw_attributes'].apply(parse_attributes)

# Extract specific attribute columns if needed
for attr in ['color', 'material', 'style', 'occasion', 'season']:
    fashion_df[attr] = fashion_df['parsed_attributes'].apply(lambda x: x.get(attr, '') if isinstance(x, dict) else '')

# Define a Schema for Phoenix
schema = px.Schema(
    embedding_feature_column_names={
        "fashion_embedding": px.EmbeddingColumnNames(
            vector_column_name="embedding",
            link_to_data_column_name="image_path",
        ),
    }
)

# Wrap into Inferences object
inferences = px.Inferences(
    dataframe=fashion_df,
    schema=schema,
    name="Fashion Images"  # Give your dataset a name
)

# Launch Phoenix!
session = px.launch_app(primary=inferences)

OMP: Info #276: omp_set_nested routine deprecated, please use omp_set_max_active_levels instead.


🌍 To view the Phoenix app in your browser, visit http://localhost:6006/
📖 For more information on how to use Phoenix, check out https://docs.arize.com/phoenix


/Users/lilysu/anaconda3/envs/py311/lib/python3.11/site-packages/sklearn/utils/deprecation.py:151: FutureWarning: 'force_all_finite' was renamed to 'ensure_all_finite' in 1.6 and will be removed in 1.8.
  warnings.warn(
/Users/lilysu/anaconda3/envs/py311/lib/python3.11/site-packages/umap/umap_.py:2462: UserWarning: n_neighbors is larger than the dataset size; truncating to X.shape[0] - 1
  warn(
/Users/lilysu/anaconda3/envs/py311/lib/python3.11/site-packages/sklearn/utils/deprecation.py:151: FutureWarning: 'force_all_finite' was renamed to 'ensure_all_finite' in 1.6 and will be removed in 1.8.
  warnings.warn(
/Users/lilysu/anaconda3/envs/py311/lib/python3.11/site-packages/umap/umap_.py:2462: UserWarning: n_neighbors is larger than the dataset size; truncating to X.shape[0] - 1
  warn(
/Users/lilysu/anaconda3/envs/py311/lib/python3.11/site-packages/sklearn/utils/deprecation.py:151: FutureWarning: 'force_all_finite' was renamed to 'ensure_all_finite' in 1.6 and will be removed in 1.8.
  